In [1]:
import numpy as np
# import matplotlib.pyplot as plt
import dedalus.public as d3
import xarray as xr
# import logging
# logger = logging.getLogger(__name__)


# Parameters
N_list = [(10**(-5))**(0.5)] #np.linspace((1e-7)**(0.5),(8e-4)**(0.5),51) #np.array([(1e-5)**(0.5)]) # stratification
delta_list = [0] #np.linspace(0, 1, 26)
theta = (1*10**(-8)/(10**(-5)))**(0.5)
f = 10**(-4)
gm = 0.9
H = 1 
H_1 = f*0.05/(gm*N_list[0]**2*theta)
lmbd = N_list[0]**2*theta*gm/f

# Basis
coord = d3.Coordinate('z')
dist = d3.Distributor(coord, dtype=np.complex128)
basis = d3.Chebyshev(coord, 32, bounds=(0, 1))

# Fields
u = dist.Field(name="u",bases=basis)
v = dist.Field(name="v",bases=basis)
w = dist.Field(name="w",bases=basis)
b = dist.Field(name="b",bases=basis)
p = dist.Field(name="p",bases=basis)
tau_1 = dist.Field(name="tau_1")
tau_2 = dist.Field(name="tau_2")
tau_3 = dist.Field(name="tau_3")
tau_4 = dist.Field(name="tau_4")
omega = dist.Field(name="omega")

# Substitutions
# z = dist.local_grid(basis)
delta = dist.Field()
N = dist.Field()
gamma = dist.Field()
Gshear = dist.Field()
Ri = dist.Field()
n = dist.Field()
alpha = dist.Field()
k = dist.Field()
lift_basis = basis.derivative_basis(1)
lift = lambda A: d3.Lift(A,lift_basis,-1)
dz = lambda A: d3.Differentiate(A, coord)
wz = dz(w)+lift(tau_1)+lift(tau_2)
pz = dz(p)+lift(tau_3)+lift(tau_4)


# Problem
problem = d3.EVP([u,v,w,b,p,tau_1,tau_2,tau_3,tau_4], eigenvalue=omega, namespace=locals())

problem.add_equation("-1j*omega*u-v+1j*Ri*k*p-alpha*b = 0")
problem.add_equation("-1j*omega*v-w+u-n*theta*w= 0")
problem.add_equation("-1j*n**2*omega*w+n*theta*v+Ri*pz-Ri*b= 0")
problem.add_equation("1j*k*u+wz=0")
problem.add_equation("-1j*omega*b + Ri**(-1)*(gamma)**(-1)*u+w= 0")

problem.add_equation("w(z=0)=0")
problem.add_equation("w(z="+str(H)+")=0")
problem.add_equation("p(z=0)=0")
problem.add_equation("p(z="+str(H)+")=0")

{'eqn': interp(<Field 5060822592>, z=1),
 'LHS': interp(<Field 5060822592>, z=1),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.complex128,
 'valid_modes': array([ True]),
 'M': 0,
 'L': interp(<Field 5060822592>, z=1),
 'domain': <dedalus.core.domain.Domain at 0x12df45950>,
 'matrix_dependence': array([ True]),
 'matrix_coupling': array([ True])}

In [4]:

# Solver
solver = problem.build_solver()
evals = []
gammas = []
k_list = np.arange(0,100,5)
time = [0] # np.arange(0,2*np.pi,0.1)
k_l = []
for ti in time:
    gammas5 = []
    evals5 = []
    for Ni in N_list:
        gammas2 = []
        N['g'] = Ni
        eval2 = []
        for deltai in delta_list:
            delta['g'] = deltai
            gamma_list = [gm]#np.linspace(gamma_lower_limit(Ni,deltai),gamma_upper_limit(Ni,deltai),21)
            eval3 = []
            gammas3 = []
            for gammai in gamma_list:
                eval4 = []
                gammas4 = []
                gamma['g'] = gammai
                Gsheari = (theta*(Ni)**2*gammai)/f
                Rii = Ni**2*(1-gammai)/Gsheari**2
                Ri['g'] = Rii
                ni = f/Gsheari
                n['g'] = ni
                alphai = Ni**2*(1-gammai)*theta/(f*Gsheari)
                alpha['g'] = alphai
                Gshear['g'] = Gsheari
                for ki in k_list:
                    k['g'] = ki
                    k_l.append(ki)
                    solver.solve_dense(solver.subproblems[0], rebuild_matrices=True)
                    omg = solver.eigenvalues
                    omg[np.isnan(omg)] = 0.
                    omg[np.isinf(omg)] = 0.
                    # omg[omg>0] = 0 #  TO AVOID SPURIOUS VALUES....
                    idx = np.sort(omg.imag)
                    sorted_evals =-1*idx[:1]
                    eval4.append(sorted_evals)
                    gammas4.append([gammai])
                eval3.append(eval4)
                gammas3.append(gammas4)
            eval2.append(eval3)
            gammas2.append(gammas3)
        evals5.append(eval2)
        gammas5.append(gammas2)
    evals.append(evals5)
    gammas.append(gammas5)
    
evals = np.array(evals)
gammas = np.array(gammas)
print(np.shape(evals))
print(np.shape(gammas))
g_index= np.linspace(0,len(gamma_list)+1,len(gamma_list))
gr_data = xr.Dataset(data_vars={"growth_rate":(["t","N","delta","gamma_index","k"],evals[:,:,:,:,:,0]),"gamma":(["t","N","delta","gamma_index","k"],gammas[:,:,:,:,:,0])},coords={"t":time,"N":N_list,"delta":delta_list,"gamma_index":g_index,"k":k_l})
# gr_data.to_netcdf("SI_debug_non_dim_10.nc")

2024-11-29 22:49:38,648 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.5e+01/s
2024-11-29 22:49:38,729 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.5e+01/s
2024-11-29 22:49:38,853 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.5e+01/s
2024-11-29 22:49:38,979 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.8e+01/s
2024-11-29 22:49:39,095 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.8e+01/s
2024-11-29 22:49:39,213 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.6e+01/s
2024-11-29 22:49:39,338 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.7e+01/s
2024-11-29 22:49:39,461 subsystems 0/1 INFO :: Building subproblem matrices 

In [6]:
gr_data.values

<bound method Mapping.values of <xarray.Dataset> Size: 512B
Dimensions:      (t: 1, N: 1, delta: 1, gamma_index: 1, k: 20)
Coordinates:
  * t            (t) int64 8B 0
  * N            (N) float64 8B 0.003162
  * delta        (delta) int64 8B 0
  * gamma_index  (gamma_index) float64 8B 0.0
  * k            (k) int64 160B 0 5 10 15 20 25 30 35 ... 65 70 75 80 85 90 95
Data variables:
    growth_rate  (t, N, delta, gamma_index, k) float64 160B -0.0 -0.0 ... -0.0
    gamma        (t, N, delta, gamma_index, k) float64 160B 0.9 0.9 ... 0.9 0.9>